# Anti Money Laundering Detection with GNN node classification
### This notenook includes GNN model training and dataset implementation with PyG library. In this example, we used HI-Small_Trans.csv as our dataset for training and testing.  

In [ ]:
import datetime
import os
from typing import Callable, Optional
import pandas as pd
from sklearn import preprocessing
import numpy as np
import torch

from torch_geometric.data import (
    Data,
    InMemoryDataset
)

pd.set_option('display.max_columns', None)
path = 'data/raw/HI-Small_Trans.csv'
df = pd.read_csv(path)

# Data visualization and possible feature engineering
Let's look into the dataset

In [ ]:
print(df.head())

After the viewing the dataframe, we suggest that we can extract all accounts from receiver and payer among all transcation for sorting the suspicious accounts. We can transform the whole dataset into node classification problem by considering accounts as nodes while transcation as edges.

The object columns should be encoded into classes with sklearn LabelEncoder.

In [ ]:
print(df.dtypes)

Check if there are any null values

In [ ]:
print(df.isnull().sum())

There are two columns representing paid and received amount of each transcation, wondering if it is necessary to split the amount into two columns when they shared the same value, unless there are transcation fee/transcation between different currency. Let's find out 

In [ ]:
print('Amount Received equals to Amount Paid:')
print(df['Amount Received'].equals(df['Amount Paid']))
print('Receiving Currency equals to Payment Currency:')
print(df['Receiving Currency'].equals(df['Payment Currency']))

It seens involved the transcations between different currency, let's print it out

In [ ]:
not_equal1 = df.loc[~(df['Amount Received'] == df['Amount Paid'])]
not_equal2 = df.loc[~(df['Receiving Currency'] == df['Payment Currency'])]
print(not_equal1)
print('---------------------------------------------------------------------------')
print(not_equal2)

The size of two df shows that there are transcation fee and transcation between different currency, we cannot combine/drop the amount columns.

As we are going to encode the columns, we have to make sure that the classes of same attribute are aligned.
Let's check if the list of Receiving Currency and Payment Currency are the same

In [ ]:
print(sorted(df['Receiving Currency'].unique()))
print(sorted(df['Payment Currency'].unique()))

# Data Preprocessing
### We will show the functions used in the PyG dataset first, dataset and model training will be provided in bottom section

In the data preprocessing, we perform below transformation:  
1. Transform the Timestamp with min max normalization.  
2. Create unique ID for each account by adding bank code with account number.  
3. Create receiving_df with the information of receiving accounts, received amount and currency
4. Create paying_df with the information of payer accounts, paid amount and currency
5. Create a list of currency used among all transactions
6. Label the 'Payment Format', 'Payment Currency', 'Receiving Currency' by classes with sklearn LabelEncoder


In [ ]:
def df_label_encoder(df, columns):
        le = preprocessing.LabelEncoder()
        for i in columns:
            df[i] = le.fit_transform(df[i].astype(str))
        return df

def preprocess(df):
        df = df_label_encoder(df,['Payment Format', 'Payment Currency', 'Receiving Currency'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
        df['Timestamp'] = (df['Timestamp']-df['Timestamp'].min())/(df['Timestamp'].max()-df['Timestamp'].min())

        df['Account'] = df['From Bank'].astype(str) + '_' + df['Account']
        df['Account.1'] = df['To Bank'].astype(str) + '_' + df['Account.1']
        df = df.sort_values(by=['Account'])
        receiving_df = df[['Account.1', 'Amount Received', 'Receiving Currency']]
        paying_df = df[['Account', 'Amount Paid', 'Payment Currency']]
        receiving_df = receiving_df.rename({'Account.1': 'Account'}, axis=1)
        currency_ls = sorted(df['Receiving Currency'].unique())

        return df, receiving_df, paying_df, currency_ls

Let's have a look of processed df

In [ ]:
df, receiving_df, paying_df, currency_ls = preprocess(df = df)
print(df.head())

paying df and receiving df:

In [ ]:
print(receiving_df.head())
print(paying_df.head())

currency_ls:

In [ ]:
print(currency_ls)

We would like to extract all unique accounts from payer and receiver as node of our graph. It includes the unique account ID, Bank code and the label of 'Is Laundering'.  
In this section, we consider both payer and receiver involved in a illicit transaction as suspicious accounts, we will label both accounts with 'Is Laundering' == 1.

In [ ]:
def get_all_account(df):
        ldf = df[['Account', 'From Bank']]
        rdf = df[['Account.1', 'To Bank']]
        suspicious = df[df['Is Laundering']==1]
        s1 = suspicious[['Account', 'Is Laundering']]
        s2 = suspicious[['Account.1', 'Is Laundering']]
        s2 = s2.rename({'Account.1': 'Account'}, axis=1)
        suspicious = pd.concat([s1, s2], join='outer')
        suspicious = suspicious.drop_duplicates()

        ldf = ldf.rename({'From Bank': 'Bank'}, axis=1)
        rdf = rdf.rename({'Account.1': 'Account', 'To Bank': 'Bank'}, axis=1)
        df = pd.concat([ldf, rdf], join='outer')
        df = df.drop_duplicates()

        df['Is Laundering'] = 0
        df.set_index('Account', inplace=True)
        df.update(suspicious.set_index('Account'))
        df = df.reset_index()
        return df

Take a look of the account list:

In [ ]:
accounts = get_all_account(df)
print(accounts.head())

# Node features
For node features, we would like to aggregate the mean of paid and received amount with different types of currency as the new features of each node. 

In [ ]:
def paid_currency_aggregate(currency_ls, paying_df, accounts):
        for i in currency_ls:
            temp = paying_df[paying_df['Payment Currency'] == i]
            accounts['avg paid '+str(i)] = temp['Amount Paid'].groupby(temp['Account']).transform('mean')
        return accounts

def received_currency_aggregate(currency_ls, receiving_df, accounts):
    for i in currency_ls:
        temp = receiving_df[receiving_df['Receiving Currency'] == i]
        accounts['avg received '+str(i)] = temp['Amount Received'].groupby(temp['Account']).transform('mean')
    accounts = accounts.fillna(0)
    return accounts

Now we can define the node attributes by the bank code and the mean of paid and received amount with different types of currency.

In [ ]:
def get_node_attr(currency_ls, paying_df,receiving_df, accounts):
        node_df = paid_currency_aggregate(currency_ls, paying_df, accounts)
        node_df = received_currency_aggregate(currency_ls, receiving_df, node_df)
        node_label = torch.from_numpy(node_df['Is Laundering'].values).to(torch.float)
        node_df = node_df.drop(['Account', 'Is Laundering'], axis=1)
        node_df = df_label_encoder(node_df,['Bank'])
#         node_df = torch.from_numpy(node_df.values).to(torch.float)  # comment for visualization
        return node_df, node_label

Take a look of node_df:

In [ ]:
node_df, node_label = get_node_attr(currency_ls, paying_df,receiving_df, accounts)
print(node_df.head())

# Edge features
In terms of edge features, we would like to conside each transcation as edges.  
For edge index, we replace all account with index and stack into a list with size of [2, num of transcation]  
For edge attributes, we used 'Timestamp', 'Amount Received', 'Receiving Currency', 'Amount Paid', 'Payment Currency' and 'Payment Format'


In [ ]:
def get_edge_df(accounts, df):
        accounts = accounts.reset_index(drop=True)
        accounts['ID'] = accounts.index
        mapping_dict = dict(zip(accounts['Account'], accounts['ID']))
        df['From'] = df['Account'].map(mapping_dict)
        df['To'] = df['Account.1'].map(mapping_dict)
        df = df.drop(['Account', 'Account.1', 'From Bank', 'To Bank'], axis=1)

        edge_index = torch.stack([torch.from_numpy(df['From'].values), torch.from_numpy(df['To'].values)], dim=0)

        df = df.drop(['Is Laundering', 'From', 'To'], axis=1)

#         edge_attr = torch.from_numpy(df.values).to(torch.float)  # comment for visualization

        edge_attr = df  # for visualization
        return edge_attr, edge_index

edge_attr:

In [ ]:
edge_attr, edge_index = get_edge_df(accounts, df)
print(edge_attr.head())

edge_index:

In [ ]:
print(edge_index)

# Final code 
### Below we will show the final code for model.py, train.py and dataset.py

# Model Architecture
In this section, we used Graph Attention Networks as our backbone model.  
The model built with two GATConv layers followed by a linear layer with sigmoid outout for classification

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv, Linear

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * heads, int(hidden_channels/4), heads=1, concat=False, dropout=0.6)
        self.lin = Linear(int(hidden_channels/4), out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, edge_index, edge_attr):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv2(x, edge_index, edge_attr))
        x = self.lin(x)
        x = self.sigmoid(x)
        
        return x

# PyG InMemoryDataset
Finally we can build the dataset with above functions

In [ ]:
class AMLtoGraph(InMemoryDataset):

    def __init__(self, root: str, edge_window_size: int = 10,
                 transform: Optional[Callable] = None,
                 pre_transform: Optional[Callable] = None):
        self.edge_window_size = edge_window_size
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)

    @property
    def raw_file_names(self) -> str:
        return 'HI-Small_Trans.csv'

    @property
    def processed_file_names(self) -> str:
        return 'data.pt'

    @property
    def num_nodes(self) -> int:
        return self._data.edge_index.max().item() + 1

    def df_label_encoder(self, df, columns):
        le = preprocessing.LabelEncoder()
        for i in columns:
            df[i] = le.fit_transform(df[i].astype(str))
        return df


    def preprocess(self, df):
        df = self.df_label_encoder(df,['Payment Format', 'Payment Currency', 'Receiving Currency'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
        df['Timestamp'] = (df['Timestamp']-df['Timestamp'].min())/(df['Timestamp'].max()-df['Timestamp'].min())

        df['Account'] = df['From Bank'].astype(str) + '_' + df['Account']
        df['Account.1'] = df['To Bank'].astype(str) + '_' + df['Account.1']
        df = df.sort_values(by=['Account'])
        receiving_df = df[['Account.1', 'Amount Received', 'Receiving Currency']]
        paying_df = df[['Account', 'Amount Paid', 'Payment Currency']]
        receiving_df = receiving_df.rename({'Account.1': 'Account'}, axis=1)
        currency_ls = sorted(df['Receiving Currency'].unique())

        return df, receiving_df, paying_df, currency_ls

    def get_all_account(self, df):
        ldf = df[['Account', 'From Bank']]
        rdf = df[['Account.1', 'To Bank']]
        suspicious = df[df['Is Laundering']==1]
        s1 = suspicious[['Account', 'Is Laundering']]
        s2 = suspicious[['Account.1', 'Is Laundering']]
        s2 = s2.rename({'Account.1': 'Account'}, axis=1)
        suspicious = pd.concat([s1, s2], join='outer')
        suspicious = suspicious.drop_duplicates()

        ldf = ldf.rename({'From Bank': 'Bank'}, axis=1)
        rdf = rdf.rename({'Account.1': 'Account', 'To Bank': 'Bank'}, axis=1)
        df = pd.concat([ldf, rdf], join='outer')
        df = df.drop_duplicates()

        df['Is Laundering'] = 0
        df.set_index('Account', inplace=True)
        df.update(suspicious.set_index('Account'))
        df = df.reset_index()
        return df
    
    def paid_currency_aggregate(self, currency_ls, paying_df, accounts):
        for i in currency_ls:
            temp = paying_df[paying_df['Payment Currency'] == i]
            accounts['avg paid '+str(i)] = temp['Amount Paid'].groupby(temp['Account']).transform('mean')
        return accounts

    def received_currency_aggregate(self, currency_ls, receiving_df, accounts):
        for i in currency_ls:
            temp = receiving_df[receiving_df['Receiving Currency'] == i]
            accounts['avg received '+str(i)] = temp['Amount Received'].groupby(temp['Account']).transform('mean')
        accounts = accounts.fillna(0)
        return accounts

    def get_edge_df(self, accounts, df):
        accounts = accounts.reset_index(drop=True)
        accounts['ID'] = accounts.index
        mapping_dict = dict(zip(accounts['Account'], accounts['ID']))
        df['From'] = df['Account'].map(mapping_dict)
        df['To'] = df['Account.1'].map(mapping_dict)
        df = df.drop(['Account', 'Account.1', 'From Bank', 'To Bank'], axis=1)

        edge_index = torch.stack([torch.from_numpy(df['From'].values), torch.from_numpy(df['To'].values)], dim=0)

        df = df.drop(['Is Laundering', 'From', 'To'], axis=1)

        edge_attr = torch.from_numpy(df.values).to(torch.float)
        return edge_attr, edge_index

    def get_node_attr(self, currency_ls, paying_df,receiving_df, accounts):
        node_df = self.paid_currency_aggregate(currency_ls, paying_df, accounts)
        node_df = self.received_currency_aggregate(currency_ls, receiving_df, node_df)
        node_label = torch.from_numpy(node_df['Is Laundering'].values).to(torch.float)
        node_df = node_df.drop(['Account', 'Is Laundering'], axis=1)
        node_df = self.df_label_encoder(node_df,['Bank'])
        node_df = torch.from_numpy(node_df.values).to(torch.float)
        return node_df, node_label

    def process(self):
        df = pd.read_csv(self.raw_paths[0])
        df, receiving_df, paying_df, currency_ls = self.preprocess(df)
        accounts = self.get_all_account(df)
        node_attr, node_label = self.get_node_attr(currency_ls, paying_df,receiving_df, accounts)
        edge_attr, edge_index = self.get_edge_df(accounts, df)

        data = Data(x=node_attr,
                    edge_index=edge_index,
                    y=node_label,
                    edge_attr=edge_attr
                    )
        
        data_list = [data] 
        if self.pre_filter is not None:
            data_list = [d for d in data_list if self.pre_filter(d)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(d) for d in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# Model Training 


In [ ]:
import torch
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
from sklearn.metrics import f1_score, precision_recall_fscore_support
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = AMLtoGraph('data')
data = dataset[0]

# Create train/validation split
split = T.RandomNodeSplit(split='train_rest', num_val=0.1, num_test=0)
data = split(data)

print(f"Total nodes: {data.num_nodes}")
print(f"Total edges: {data.num_edges}")
print(f"Training nodes: {data.train_mask.sum().item()}")
print(f"Validation nodes: {data.val_mask.sum().item()}")

# Move data to device
data = data.to(device)

# Create efficient batches
def create_batches(indices, batch_size):
    batches = []
    for i in range(0, len(indices), batch_size):
        batch = indices[i:i + batch_size]
        batches.append(batch)
    return batches

batch_size = 1024  # Larger batches for efficiency
train_indices = data.train_mask.nonzero(as_tuple=False).flatten()
val_indices = data.val_mask.nonzero(as_tuple=False).flatten()

train_batches = create_batches(train_indices, batch_size)
val_batches = create_batches(val_indices, batch_size)

print(f"Training batches: {len(train_batches)}")
print(f"Validation batches: {len(val_batches)}")

# Use your existing GAT model
model = GAT(in_channels=data.num_features, hidden_channels=16, out_channels=1, heads=4)
model = model.to(device)

# Better loss function and optimizer
train_labels = data.y[data.train_mask]
class_counts = torch.bincount(train_labels.long())
class_weights = 1.0 / class_counts.float()
pos_weight = class_weights[1] / class_weights[0]
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))

optimizer = torch.optim.AdamW(model.parameters(), lr=0.003, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)

def train_epoch():
    model.train()
    total_loss = 0
    num_batches = 0
    
    # Shuffle batches
    random.shuffle(train_batches)
    
    # Process every 4th batch to speed up training
    selected_batches = train_batches[::4]  # Use every 4th batch
    
    for batch_indices in selected_batches:
        optimizer.zero_grad()
        
        # Forward pass on entire graph
        pred = model(data.x, data.edge_index, data.edge_attr)
        
        # Extract predictions for current batch
        batch_pred = pred[batch_indices]
        batch_labels = data.y[batch_indices].float().unsqueeze(1)
        
        loss = criterion(batch_pred, batch_labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    return total_loss / num_batches if num_batches > 0 else 0

@torch.no_grad()
def validate():
    model.eval()
    all_preds = []
    all_labels = []
    
    # Get full predictions once
    full_pred = model(data.x, data.edge_index, data.edge_attr)
    
    # Process validation batches
    for batch_indices in val_batches:
        batch_pred = full_pred[batch_indices]
        batch_labels = data.y[batch_indices]
        
        pred_probs = torch.sigmoid(batch_pred).squeeze()
        pred_binary = (pred_probs > 0.5).long()
        
        all_preds.extend(pred_binary.cpu().tolist())
        all_labels.extend(batch_labels.long().cpu().tolist())
    
    accuracy = sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    
    return accuracy, precision, recall, f1

# Training loop with faster convergence
epochs = 30  # Reduced epochs
best_val_f1 = 0
patience_counter = 0
patience = 5  # Reduced patience

print("Starting optimized training...")
start_time = time.time()

for epoch in range(epochs):
    epoch_start = time.time()
    train_loss = train_epoch()
    epoch_time = time.time() - epoch_start
    
    if epoch % 1 == 0:  # Validate every epoch
        val_acc, val_precision, val_recall, val_f1 = validate()
        scheduler.step(val_f1)
        
        print(f"Epoch {epoch:03d} | Time: {epoch_time:.2f}s | Train Loss: {train_loss:.4f}")
        print(f"Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f} | Val Precision: {val_precision:.4f}")
        print("-" * 50)
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

total_time = time.time() - start_time
print(f"\nTraining completed in {total_time/60:.2f} minutes")
print(f"Best validation F1: {best_val_f1:.4f}")

# Final evaluation
model.eval()
with torch.no_grad():
    full_pred = model(data.x, data.edge_index, data.edge_attr)
    
    # Training accuracy
    train_pred = torch.sigmoid(full_pred[data.train_mask]).squeeze()
    train_pred_binary = (train_pred > 0.5).long()
    train_labels = data.y[data.train_mask]
    train_accuracy = (train_pred_binary == train_labels).sum().item() / len(train_labels)
    
    # Validation accuracy
    val_pred = torch.sigmoid(full_pred[data.val_mask]).squeeze()
    val_pred_binary = (val_pred > 0.5).long()
    val_labels = data.y[data.val_mask]
    val_accuracy = (val_pred_binary == val_labels).sum().item() / len(val_labels)
    
    print(f"\nFinal Training Accuracy: {train_accuracy:.4f}")
    print(f"Final Validation Accuracy: {val_accuracy:.4f}")
    
    # Class distribution
    train_class_0 = (train_labels == 0).sum().item()
    train_class_1 = (train_labels == 1).sum().item()
    val_class_0 = (val_labels == 0).sum().item()
    val_class_1 = (val_labels == 1).sum().item()
    
    print(f"\nTraining set - Normal: {train_class_0}, Suspicious: {train_class_1}")
    print(f"Validation set - Normal: {val_class_0}, Suspicious: {val_class_1}")
    
    if min(train_class_0, train_class_1) > 0:
        train_ratio = max(train_class_0, train_class_1) / min(train_class_0, train_class_1)
        print(f"Training imbalance ratio: {train_ratio:.2f}:1")
    
    if min(val_class_0, val_class_1) > 0:
        val_ratio = max(val_class_0, val_class_1) / min(val_class_0, val_class_1)
        print(f"Validation imbalance ratio: {val_ratio:.2f}:1")

print("\nOptimizations applied:")
print("✓ Larger batch size (1024)")
print("✓ Sparse batch training (every 4th batch)")
print("✓ Better optimizer (AdamW)")
print("✓ Weighted loss for class imbalance")
print("✓ Early stopping")
print("✓ Gradient clipping")

# Future Work
In this notebook, we performed the node classification with GAT and the result accuracy looks satisfied.  
However, it may due to highly imbalance data of the dataset. It is suggested that balance the class of 1 and 0 in the data preprocessing. It is expected that the accuracy will dropped a little bit after balancing the data.  We will keep exploring to see if there are any other models give better performance, such as other traditional regression/classifier model.

Save model

In [ ]:

torch.save({
    'model_state_dict': model.state_dict(),
    'epoch': epoch,
    'optimizer_state_dict': optimizer.state_dict(),
}, 'best_model.pth')

## Reference
Some of the feature engineering of this repo are referenced to below papers, highly recommend to read:
1. [Weber, M., Domeniconi, G., Chen, J., Weidele, D. K. I., Bellei, C., Robinson, T., & Leiserson, C. E. (2019). Anti-money laundering in bitcoin: Experimenting with graph convolutional networks for financial forensics. arXiv preprint arXiv:1908.02591.](https://arxiv.org/pdf/1908.02591.pdf)
2. [Johannessen, F., & Jullum, M. (2023). Finding Money Launderers Using Heterogeneous Graph Neural Networks. arXiv preprint arXiv:2307.13499.](https://arxiv.org/pdf/2307.13499.pdf)